# Segmenting and Clustering Neighborhoods in Toronto

## Submission for Question 3

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import xml
import numpy as np

### Pre-processing: Repeat the steps from Q1 and Q2 to create a merged dataframe

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url)

In [3]:
tbl_tag = soup.find('table')
td_tag = tbl_tag.find_all('td')

In [4]:
# store values in lists
postalcodes = []
boroughs = []
neighborhoods = []

# note that relevant info td tags are in group of 3 tags.
indexes = np.arange(len(td_tag), step=3)

for idx in indexes:
    postalcodes.append(td_tag[idx].text.strip())
    boroughs.append(td_tag[idx+1].text.strip())
    neighborhoods.append(td_tag[idx+2].text.strip())

In [5]:
df = pd.DataFrame(data=[postalcodes, boroughs, neighborhoods])
df = df.transpose()

In [6]:
# Replace column names. Remove boroughs that are not assigned.
df.columns = ['PostalCode', 'Borough', 'Neighborhood']

In [7]:
df.drop( df[ df['Borough'] == "Not assigned" ].index , inplace=True)

In [8]:
# NA_indexes = df['Neighborhood'] == "Not assigned"
# df["Neighborhood"][NA_indexes] = df["Borough"][NA_indexes]

In [9]:
df_grouped = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [10]:
# Geospatial_Coordinates.csv was downloaded from http://cocl.us/Geospatial_data
df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [11]:
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']

In [12]:
df_merged = pd.merge(df_grouped, df_geo, on=['PostalCode'], how='inner')
df_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


## Answer to Question 3

In [13]:
import folium
from geopy.geocoders import Nominatim

### Toronto

In [25]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('Coordinates of Toronto: {}, {}'.format(latitude, longitude))

/Users/twh/miniconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


Coordinates of Toronto: 43.653963, -79.387207


In [26]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lon, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_clusters)  

map_clusters